In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [13]:
df_emissions_raw = pd.read_csv('./data/Emissions_de_GEH_a_Catalunya_20250127.csv', sep=',', decimal='.')
df_emissions_raw.sample(5)

,ANY,PROVINCIA,ACTIVITAT,IPCC_SECTOR,IPCC_DIVISIO,IPCC_CLASSE,IPCC_SUBCLASSE,NOM_SECTOR,NOM_DIVISIO,NOM_CLASSE,NOM_SUBCLASSE,CONTAMINANT,UNITATS,CO2EQ,EMISSIO GAS
23015,2020,TARRAGONA,2G1a,2,G,1.0,a,Processos Industrials,Producció i ús d'altres productes,Equips elèctrics,Etapa de fabricació,SF6,Gg,0.388374,0.000017
11489,2022,GIRONA,1A2d,1,A,2.0,d,Processament de l'energia,Activitats de combustió,Indústries manufactureres i de la construcció,"Pasta de paper, paper i impressió",CH4,Gg,4.823343,0.172262
3095,2007,BARCELONA,3B11,3,B,1.0,1,Agricultura,Gestió de fems,Emissions de CH4,Bestiar boví,CH4,Gg,45.073192,1.609757
4363,2014,BARCELONA,2F1c,2,F,1.0,c,Processos Industrials,Usos de productes com a substituts per a les s...,Refrigeració i aire condicionat,Refrigeració industrial,HFC-134a,Gg,125.344928,0.096419
6464,1992,GIRONA,4D22,4,D,2.0,2,"Ús del sòl, canvis d'ús del sòl i silvicultura",Aiguamolls,Terres convertides en aiguamolls,Terres convertides en terres inundades,CO2,Gg,0.180750,0.180750


In [7]:
df_emissions_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ANY             23486 non-null  int64  
 1   PROVINCIA       23486 non-null  object 
 2   ACTIVITAT       23486 non-null  object 
 3   IPCC_SECTOR     23486 non-null  int64  
 4   IPCC_DIVISIO    23486 non-null  object 
 5   IPCC_CLASSE     23222 non-null  float64
 6   IPCC_SUBCLASSE  21795 non-null  object 
 7   NOM_SECTOR      23486 non-null  object 
 8   NOM_DIVISIO     23486 non-null  object 
 9   NOM_CLASSE      23486 non-null  object 
 10  NOM_SUBCLASSE   23420 non-null  object 
 11  CONTAMINANT     23486 non-null  object 
 12  UNITATS         23486 non-null  object 
 13  CO2EQ           23486 non-null  float64
 14  EMISSIO GAS     23486 non-null  float64
dtypes: float64(3), int64(2), object(10)
memory usage: 2.7+ MB


In [12]:
df_emissions = df_emissions_raw.groupby(['ANY', 'CONTAMINANT'])['CO2EQ'].sum().unstack()

df_emissions['OTHER GASES'] = df_emissions.drop(columns=['CO2', 'CH4', 'N2O'], errors='ignore').sum(axis=1)
df_emissions['EMISSIONS TOTAL'] = df_emissions[['CO2', 'CH4', 'N2O', 'OTHER GASES']].sum(axis=1)
df_emissions = df_emissions[['CO2', 'CH4', 'N2O', 'OTHER GASES', 'EMISSIONS TOTAL']]

df_emissions.sample(5)

CONTAMINANT,CO2,CH4,N2O,OTHER GASES,EMISSIONS TOTAL
ANY,,,,,
1999,31738.969721,6987.143100,1503.253337,5430.770314,45660.136473
2007,40963.458478,6427.388533,1370.020294,2859.053866,51619.921170
1995,32196.421925,6541.270308,1432.630040,3963.214530,44133.536804
2003,37134.378484,6925.777059,1453.209834,2115.066988,47628.432365
2013,26555.819804,5818.983509,1163.452978,2482.078870,36020.335161


In [8]:
df_emissions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 1990 to 2022
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CO2              33 non-null     float64
 1   CH4              33 non-null     float64
 2   N2O              33 non-null     float64
 3   OTHER GASES      33 non-null     float64
 4   EMISSIONS TOTAL  33 non-null     float64
dtypes: float64(5)
memory usage: 1.5 KB


In [11]:
df_emissions.describe()

CONTAMINANT,CO2,CH4,N2O,OTHER GASES,EMISSIONS TOTAL
count,33.000000,33.000000,33.000000,33.000000,33.000000
mean,31325.554548,6307.604570,1317.671031,2479.800675,41430.630825
std,4803.284524,490.917798,114.603041,1309.348651,5522.160232
min,25065.077518,5406.389064,1163.452978,819.979776,34053.042761
25%,27454.146921,5925.402693,1220.261155,1426.725346,36365.007181
50%,30145.129078,6200.813051,1280.864603,2401.912087,41381.849333
75%,34086.952676,6645.697158,1432.518715,2859.053866,45141.408648
max,41759.780690,7389.716448,1551.455543,5944.170230,51619.921170
